In [1]:
import numpy as np
import numpy.matlib
import matplotlib
%matplotlib inline
import h5py
import matplotlib.pyplot as plt
import sys,os
import importlib
import sklearn
import sklearn.decomposition
import sklearn.preprocessing
import datetime
import seaborn as sns
import pandas as pd
import scipy
import scipy.spatial
import dReLU_Regression
import Gaussian_Regression
import scipy.fftpack
from tqdm import tqdm
import scipy.io
import pickle
username = os.getlogin()
sys.path.append(f'/home/{username}/repos/dnp-code/')
import rbm_pipeline_functions as pf
from fishualizer_utilities import Zecording
sys.path.append(f'/home/{username}/repos/dnp-code/PGM3_correct/source/')
import rbm, utilities
print(pf.__file__)


Bad key "text.kerning_factor" on line 4 in
/home/thijs/.conda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/home/thijs/.conda/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/thijs/.conda/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a de

/home/thijs/repos/dnp-code/rbm_pipeline_functions.py


In [ ]:
assert pf.__file__ == f'/home/{username}/repos/dnp-code/rbm_pipeline_functions.py'

In [6]:
## Plot settings

plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.style'] = 'normal'
plt.rcParams['lines.linewidth'] = 3
dr_colors = {'glm': '#0099cc', #'#008B8B', 
             'pca': '#808000', 'rbm': '#800080', 'fa': 'red', 'ica':'#157bf9'}
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color'] # default color cycle

dr_names = {'rbm': 'cRBM', 'RBM': 'cRBM', 'pca': 'PCA', 'PCA': 'PCA',
            'covariance': 'covariance', 'correlation': 'correlation'}
# dr_legend = {'pca': 'PCA', 'rbm': 'RBM', 'fa': 'FA', 'ica': 'ICA', 'glm': 'GLM'}

In [ ]:
if username == 'thijs':
    base_path = '/home/thijs/Desktop/zf_rbm_essentials/'
else:
    base_path = '/home/tplas/data/zf/'
# base_path = '/media/thijs/hooghoudt/Zebrafish_data/spontaneous_data_guillaume/'
data_sets = {#'20180912-Run01': '20180912_Run01Tset=.h5'}
            '20180912-Run01': '20180912_Run01_spontaneous_rbm2.h5'}#,
test_segs = '267'
train_inds_path=f'/home/{username}/repos/dnp-code/train_test_inds/20180912-Run01/train_test_inds__test_segs_{test_segs}_nseg10.pkl'  # HARD SET TO 10TH PERCENTILE OF 20180912-RUN01 (TEST SEGS 267)


In [7]:
## Load data
recordings = {}
for data_set, data_path in data_sets.items():
    recordings[data_set] = Zecording(path=base_path + data_path, kwargs={'ignorelags': True,
                                              'forceinterpolation': False,
                                              'ignoreunknowndata': True,# 'parent': self,
                                              'loadram': True})  # load data
rec = recordings[list(data_sets.keys())[0]]
# rec = recordings['2019-03-26(Run09)']
print(rec)
regions = {#'rh1': np.array([218]), 'rhall': np.array([113]),
          'wb': np.arange(294)}
selected_neurons = {}
n_sel_cells = {}
train_data = {}
test_data = {}
full_data = {}

dict_tt_inds = pickle.load(open(train_inds_path, 'rb'))  # load dictionary with training indices
train_inds = dict_tt_inds['train_inds']  # load training inds, note that: # test_inds = dict_tt_inds['test_inds']
test_inds = dict_tt_inds['test_inds']
print(f'len test inds {len(test_inds)}')
for ir in list(regions.keys()):
    selected_neurons[ir] = np.unique(scipy.sparse.find(rec.labels[:, regions[ir]])[0])
    assert rec.spikes.shape[0] > rec.spikes.shape[1]
    train_data[ir] = rec.spikes[selected_neurons[ir], :][:, train_inds]
    test_data[ir] = rec.spikes[selected_neurons[ir], :][:, test_inds]
    n_sel_cells[ir] = len(selected_neurons[ir])
    full_data[ir] = rec.spikes[selected_neurons[ir], :]

/home/thijs/.conda/envs/py37/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
baseline with shape (5553, 54334) is not recognized, so it cannot be loaded.
drifts with shape (5553, 2) is not recognized, so it cannot be loaded.
inferredspikes with shape (5553, 54334) is not recognized, so it cannot be loaded.
ljpcoordinates with shape (3, 54334) is not recognized, so it cannot be loaded.
segmentation with shape (30, 598, 1280) is not recognized, so it cannot be loaded.
temporalmean with shape (30, 598, 1280) is not recognized, so it cannot be loaded.
rawsignal with shape (5553, 54334) is not recognized, so it cannot be loaded.
trace with shape (5553, 1) is not recognized, so it cannot be loaded.
metadata with shape (1, 1) is not recognized, so it cannot be loaded.


Recording from /home/thijs/Desktop/zf_rbm_essentials/20180912_Run01_spontaneous_rbm2.h5
len test inds 1665


In [4]:
inds_wb = np.unique(scipy.sparse.find(rec.labels[:, np.arange(294)])[0])